In [ ]:
from agno.agent import Agent
from agno.team import Team
from ddgs import DDGS
import os
os.environ["OPENAI_API_KEY"] = "🖕"
os.environ["OPENAI_BASE_URL"] = "https://api.proxyapi.ru/openai/v1"


In [11]:
def flow_search(zapros):
    ddg = DDGS()
    rezultaty = list(ddg.text(zapros, max_results=5))
    for i, rez in enumerate(rezultaty, 1):
        print(rez['title'])
        print(rez['href'])
        print(rez['body'][:100])
    return rezultaty
flow_results = flow_search("real name Инстасамка")

Инстасамка и её орех (Дарья Зотеева) — Видео от Егор Литвиненко
https://yandex.ru/video/preview/9538951173732545228
Инстасамка и её орех (Дарья Зотеева) — Видео от Егор Литвиненко. 76,9K76,9 тысяч просмотров.
INSTASAMKA - Personality (prod.realmoneyken) СЛИВ... - YouTube
https://www.youtube.com/watch?v=tqAJJl3Ni5Y
INSTASAMKA - ПРЕМИУМ ПОДПИСКА (prod.realmoneyken) СЛИВ ТРЕКА. moneybarbie. 331 просмотр • 7 дней наз
Инстасамка и олег moneyken watch online
https://ukdevilz.com/watch/-193480880_456239070
Слив инстасамки инстасосалки instasamka инстасамка и олег слив порно секс минет прон porn. Инстасамк
Инстасамка - Биография и дискография, все альбомы и песни
https://minatrix.fm/artists/name/instasamka/
InstaSamka (Дарья Зотеева) — российская рэп-исполнительница и блогер, ставшая известной благодаря эп
INSTASAMKA Age, Net Worth, Height, Wiki, Family
https://wikiabio.com/instasamka/
Real Name . INSTASAMKA Biography. Pop singer and social media star who is finest identified for her 


In [13]:
def search_tool(zapros: str) -> str:
    ddg = DDGS()
    rezultaty = list(ddg.text(zapros, max_results=3))
    output = ""
    for i, r in enumerate(rezultaty, 1):
        output += f"{r['title']}{r['href']}"
    return output
def agent_search(zapros):
    agent = Agent(
        name="Searcher",
        tools=[search_tool],
        instructions="you are a search agent. Use search_tool to find information.",
        markdown=False,
        model="openai:gpt-4o-mini"
    )
    otvet = agent.run(zapros)
    print(otvet.content)
    return otvet.content
agent_results = agent_search("find real name Инстасамка")

Инстасамка — это псевдоним Дарьи Зотеевой.


In [ ]:
def search_web(zapros: str) -> str:
    ddg = DDGS()
    rezultaty = list(ddg.text(zapros, max_results=10))
    output = ""
    for r in rezultaty:
        output += f"{r['title']}{r['href']}{r['body'][:100]}"
    return output

def team_search(zapros):
    searcher = Agent(
        name="Searcher",
        role="Search the web for information",
        tools=[search_web],
        markdown=False,
        model="openai:gpt-4o-mini"
    )
    analyzer = Agent(
        name="Analyzer",
        role="Analyze search results and extract key information",
        markdown=False,
        model="openai:gpt-4o-mini"
    )
    team = Team(
        members=[searcher, analyzer],
        name="Search Team",
        model="openai:gpt-4o-mini"
    )
    otvet = team.run(zapros)
    print(otvet.content)
    return otvet

team_results = team_search("find real name Инстасамка")

The real name of the Russian artist Инстасамка is Дарья Еропкина (Daria Yeropkina).
